# Analiza współczynnika uwarunkowania macierzy
Paweł Skierkowski

Wyznaczanie współczynnika uwarunkowania macierzy metodą potęgową

In [281]:
import numpy as npfrom scipy.linalg import hilbert

## Zadanie 1:
### Metoda potęgowa:

In [244]:
def m_potegowa(C):
    t = np.ones(C.shape[0])
    t = t / np.linalg.norm(t, ord=np.inf)
    l_old = 0

    while True:
        t = C @ t
        t = t / np.linalg.norm(t, ord=np.inf)
        l_new = (t.T @ C @ t)/(t.T@t)
        error = abs(l_new - l_old)
        if error < 1e-6:
            break
        l_old = l_new
        
    return l_new


### Wyznaczanie maksymalnej wartości własnej:

In [247]:
def lam_max(C):
    return m_potegowa(C)

### Wyznaczanie minimalnej wartości własnej stosując przesunięcie:

In [294]:
def lam_min(A,l):
    B=A-l*np.eye(A.shape[0])
    l_min=m_potegowa(B)
    return l_min+l

### Wyznaczanie współczynnika uwarunkowania:

In [253]:
def cond(C):
    A = C.T @ C
    lam_max_val = lam_max(A)
    lam_min_val = lam_min(A, lam_max_val)
    return np.sqrt(lam_max_val / lam_min_val)
    

### Porównanie wyników z wbudowaną funkcją:

In [256]:
def comp(A):
    implemented = cond(A)
    included = np.linalg.cond(A)
    difference = abs(implemented - included) / included * 100
    return implemented, included, difference

### Wyznaczenie współczynnika oraz różnic pomiedzy metodami na podstawie macierzy C z podręcznika

In [259]:
C = np.array([
    [4,2,-5,2],
    [1,5,3,9],
    [2,2,5,-7],
    [1,4,-1,1]
])
implemented,included,difference=comp(C)

print(f"Wyliczony współczynnik: {implemented}")
print(f"Wbudowana funckja: {included}")
print(f"Różnica pomiędzy metodami: {difference:.8f}%")

Wyliczony współczynnik: 6.43169731895808
Wbudowana funckja: 6.4316983882385355
Różnica pomiędzy metodami: 0.00001663%


### Podsumowanie zadania 1:
Otrzymane wyniki są niemal identyczne, różnica pomiedzy zaimplementowaną metodą m_potegowa, a metodą z biblioteki numpy jest nieznaczna dla podanej macierzy.

## Zadanie 2:
### Wpływ właściwości macierzy (rodzaj, rozmiar) na efektywność metody:

In [291]:
R = np.random.rand(50, 50)
A_sym = (R + R.T) / 2

matrices = {
    "Symetryczna 4x4": np.array([
        [4, 1, 2, 0],
        [1, 3, 0, 1],
        [2, 0, 5, 1],
        [0, 1, 1, 2]
    ]),
    "Niesymetryczna 4x4": np.array([
        [2, -1, 0, 3],
        [0, 1, 4, 0],
        [1, 0, 2, -1],
        [1, 1, 1, 1]
    ]),
    "Losowa 5x5": np.random.rand(5, 5),
    "Diagonalna 4x4": np.diag([1, 10, 100, 1000]),
    "Symetryczna 50x50": A_sym,
    "Niesymetryczna 50x50": np.random.rand(50,50),
    "Hilberta 5x5": hilbert(5),
    "Hilberta 50x50": hilbert(50)
}
for name, A in matrices.items():
    implemented, included, diff = comp(A)
    print(f"\nMacierz: {name}")
    print(f"  Wyliczony cond:    {implemented:.8f}")
    print(f"  Wbudowana cond:    {included:.8f}")
    print(f"  Różnica:           {diff:.6f}%")


Macierz: Symetryczna 4x4
  Wyliczony cond:    8.44118701
  Wbudowana cond:    8.44120150
  Różnica:           0.000172%

Macierz: Niesymetryczna 4x4
  Wyliczony cond:    4.96799495
  Wbudowana cond:    4.96801960
  Różnica:           0.000496%

Macierz: Losowa 5x5
  Wyliczony cond:    26.06577106
  Wbudowana cond:    26.13153834
  Różnica:           0.251678%

Macierz: Diagonalna 4x4
  Wyliczony cond:    997.48510786
  Wbudowana cond:    1000.00000000
  Różnica:           0.251489%

Macierz: Symetryczna 50x50
  Wyliczony cond:    157.31879812
  Wbudowana cond:    660.02383851
  Różnica:           76.164679%

Macierz: Niesymetryczna 50x50
  Wyliczony cond:    363.19968527
  Wbudowana cond:    7057.05480733
  Różnica:           94.853382%

Macierz: Hilberta 5x5
  Wyliczony cond:    126.73711652
  Wbudowana cond:    476607.25024100
  Różnica:           99.973408%

Macierz: Hilberta 50x50
  Wyliczony cond:    79.14662308
  Wbudowana cond:    17175883540120668160.00000000
  Różnica:       

### Wnioski:
Dla macierzy niewielkich obie metody dają podobne wyniki. 

Dla macierzy o większych rozmiarach (50x50) różnica staję się zdecydowaie większa, zaimplementowana metoda działa jednak lepiej dla macierzy symetrycznych niż niesymetrycznych.

Dla macierzy Hilberta - zarówno dużej jak i małej - zaimplementowana metoda potęgowa nie pozwala dokładnie wyznaczyć uwarunkowania. Macierz Hilberta jest bardzo źle uwarunkowana. 